In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
#挂载
!mkdir -p drive
!google-drive-ocamlfuse drive


In [0]:
#进入创建的CAR文件夹
cd drive/CAR

In [0]:
#安装keras
!pip install -q keras

In [0]:

"""
生成车牌的一个类  GenPlate
要将NoPlates、font、image放在CAR这个目录下（当前目录）
"""
import matplotlib.pyplot as plt
import numpy as np
import random
import PIL
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
import cv2
import numpy as np
import os
from math import *
import sys
 
chars = ["京", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "皖", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂",
             "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A",
             "B", "C", "D", "E", "F", "G", "H", "J", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "U", "V", "W", "X",
             "Y", "Z"
             ]



index = {"京": 0, "沪": 1, "津": 2, "渝": 3, "冀": 4, "晋": 5, "蒙": 6, "辽": 7, "吉": 8, "黑": 9, "苏": 10, "浙": 11, "皖": 12,
         "闽": 13, "赣": 14, "鲁": 15, "豫": 16, "鄂": 17, "湘": 18, "粤": 19, "桂": 20, "琼": 21, "川": 22, "贵": 23, "云": 24,
         "藏": 25, "陕": 26, "甘": 27, "青": 28, "宁": 29, "新": 30, "0": 31, "1": 32, "2": 33, "3": 34, "4": 35, "5": 36,
         "6": 37, "7": 38, "8": 39, "9": 40, "A": 41, "B": 42, "C": 43, "D": 44, "E": 45, "F": 46, "G": 47, "H": 48,"I":49,
         "J": 50, "K": 51, "L": 52, "M": 53, "N": 54, "O":55,"P": 56, "Q": 57, "R": 58, "S": 59, "T": 60, "U": 61, "V": 62,
         "W": 63, "X": 64, "Y": 65, "Z": 66}

def AddSmudginess(img, Smu):
    rows = r(Smu.shape[0] - 50)
    cols = r(Smu.shape[1] - 50)
    adder = Smu[rows:rows + 50, cols:cols + 50]
    adder = cv2.resize(adder, (50, 50))
    #adder = cv2.bitwise_not(adder)
    img = cv2.resize(img,(50,50))
    img = cv2.bitwise_not(img)
    img = cv2.bitwise_and(adder, img)
    img = cv2.bitwise_not(img)
    return img
 
def rot(img,angel,shape,max_angel):
    """
      添加透视畸变
      """
    size_o = [shape[1],shape[0]]
    size = (shape[1]+ int(shape[0]*cos((float(max_angel )/180) * 3.14)),shape[0])
    interval = abs( int( sin((float(angel) /180) * 3.14)* shape[0]))
    pts1 = np.float32([[0,0],[0,size_o[1]],[size_o[0],0],[size_o[0],size_o[1]]])
    if(angel>0):
        pts2 = np.float32([[interval,0],[0,size[1]  ],[size[0],0  ],[size[0]-interval,size_o[1]]])
    else:
        pts2 = np.float32([[0,0],[interval,size[1]  ],[size[0]-interval,0  ],[size[0],size_o[1]]])
    M  = cv2.getPerspectiveTransform(pts1,pts2)
    dst = cv2.warpPerspective(img,M,size)
    return dst
 
def rotRandrom(img, factor, size):
 
    """
    添加放射畸变
    img 输入图像
    factor 畸变的参数
    size 为图片的目标尺寸
    """
    shape = size
    pts1 = np.float32([[0, 0], [0, shape[0]], [shape[1], 0], [shape[1], shape[0]]])
    pts2 = np.float32([[r(factor), r(factor)], [ r(factor), shape[0] - r(factor)], [shape[1] - r(factor),  r(factor)],
                       [shape[1] - r(factor), shape[0] - r(factor)]])
    M = cv2.getPerspectiveTransform(pts1, pts2)
    dst = cv2.warpPerspective(img, M, size)
    return dst
 
def tfactor(img):
    """
    添加饱和度光照的噪声
    """
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    hsv[:,:,0] = hsv[:,:,0]*(0.8+ np.random.random()*0.2)
    hsv[:,:,1] = hsv[:,:,1]*(0.3+ np.random.random()*0.7)
    hsv[:,:,2] = hsv[:,:,2]*(0.2+ np.random.random()*0.8)
 
    img = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    return img
 
def random_envirment(img,data_set):
    """
    添加自然环境的噪声    
    """
    index=r(len(data_set))
    env = cv2.imread(data_set[index])
    env = cv2.resize(env,(img.shape[1],img.shape[0]))
    bak = (img==0)
    bak = bak.astype(np.uint8)*255
    inv = cv2.bitwise_and(bak,env)
    img = cv2.bitwise_or(inv,img)
    return img
 
def GenCh(f,val):
    """
    生成中文字符
    """
    img=Image.new("RGB", (45,70),(255,255,255))
    draw = ImageDraw.Draw(img)
    draw.text((0, 3),val,(0,0,0),font=f)
    img =  img.resize((23,70))
    A = np.array(img)
    return A
 
def GenCh1(f,val):
    """
    生成英文字符
    """
    img=Image.new("RGB", (23,70),(255,255,255))
    draw = ImageDraw.Draw(img)
    draw.text((0, 2),val,(0,0,0),font=f) # val.decode('utf-8')
    A = np.array(img)
    return A
 
def AddGauss(img, level):
    """
    添加高斯模糊
    """ 
    return cv2.blur(img, (level * 2 + 1, level * 2 + 1))
 
def r(val):
    return int(np.random.random() * val)
 
def AddNoiseSingleChannel(single):
    """
    添加高斯噪声
    """
    diff = 255-single.max()
    noise = np.random.normal(0,1+r(6),single.shape)
    noise = (noise - noise.min())/(noise.max()-noise.min())
    noise= diff*noise
    noise= noise.astype(np.uint8)
    dst = single + noise
    return dst
 
def addNoise(img,sdev = 0.5,avg=10):
    img[:,:,0] =  AddNoiseSingleChannel(img[:,:,0])
    img[:,:,1] =  AddNoiseSingleChannel(img[:,:,1])
    img[:,:,2] =  AddNoiseSingleChannel(img[:,:,2])
    return img
 
 
class GenPlate:
 
    def __init__(self,fontCh,fontEng,NoPlates):
        self.fontC =  ImageFont.truetype(fontCh,43,0)
        self.fontE =  ImageFont.truetype(fontEng,60,0)
        self.img=np.array(Image.new("RGB", (226,70),(255,255,255)))
        self.bg  = cv2.resize(cv2.imread("./images/template.bmp"),(226,70))
        self.smu = cv2.imread("./images/smu2.jpg")
        self.noplates_path = []
        for parent,parent_folder,filenames in os.walk(NoPlates):
            for filename in filenames:
                path = parent+"/"+filename
                self.noplates_path.append(path)
 
 
    def draw(self,val):
        offset= 2
        self.img[0:70,offset+8:offset+8+23]= GenCh(self.fontC,val[0])
        self.img[0:70,offset+8+23+6:offset+8+23+6+23]= GenCh1(self.fontE,val[1])
        for i in range(5):
            base = offset+8+23+6+23+17 +i*23 + i*6
            self.img[0:70, base  : base+23]= GenCh1(self.fontE,val[i+2])
        return self.img
    
    def generate(self,text):
        if len(text) == 7:
            fg = self.draw(text) #.decode(encoding="utf-8")
            fg = cv2.bitwise_not(fg)
            com = cv2.bitwise_or(fg,self.bg)
            com = rot(com,r(60)-30,com.shape,30)
            com = rotRandrom(com,10,(com.shape[1],com.shape[0]))
            com = tfactor(com)
            com = random_envirment(com,self.noplates_path)
            com = AddGauss(com, 1+r(4))
            com = addNoise(com)
            return com
 
    def genPlateString(self,pos,val):
        '''
	    生成车牌String,存为图片
       
        '''
        plateStr = ""
        box = [0,0,0,0,0,0,0]
        if(pos!=-1):
            box[pos]=1
        for unit,cpos in zip(box,range(len(box))):
            if unit == 1:
                plateStr += val
                #print plateStr
            else:
                if cpos == 0:
                    plateStr += chars[r(31)]
                elif cpos == 1:
                    plateStr += chars[41+r(24)]                 
                else:
                    plateStr += chars[31 + r(34)]
        return plateStr
G = GenPlate("./font/platech.ttf",'./font/platechar.ttf',"./NoPlates")



In [0]:
#使用的是resnet34模型
from keras.models import Model  
from keras.layers import Input,Dense,Dropout,BatchNormalization,Conv2D,MaxPooling2D,AveragePooling2D,concatenate,Activation,ZeroPadding2D  
from keras.layers import add,Flatten   
import numpy as np  
seed = 7  
np.random.seed(seed)  

characters="京沪津渝冀晋蒙辽吉黑苏浙皖闽赣鲁豫鄂湘粤桂琼川贵云藏陕甘青宁新0123456789ABCDEFGHJKLMNPQRSTUVWXYZ"

width, height, n_len, n_class = 224, 224, 7, len(characters)

def gen(batch_size=32):
    X = np.zeros((batch_size, height, width, 3), dtype=np.uint8)
    y = [np.zeros((batch_size, n_class), dtype=np.uint8) for i in range(n_len)]
   # generator = ImageCaptcha(width=width, height=height)
    while True:
        for i in range(batch_size):
            a=G.genPlateString(-1,-1)
            b=G.generate(a)
            img = cv2.resize(b,(224,224))
            X[i] = img.astype('float32')
            for j, ch in enumerate(a):
                y[j][i, :] = 0
                y[j][i, characters.find(ch)] = 1
        yield X, y

def decode(y):
    y = np.argmax(np.array(y), axis=2)[:,0]
    return ''.join([characters[x] for x in y])
 
"""
使用者三段代码测试生成结果，以及解码结果
X, y = next(gen(1))
plt.imshow(X[0])
print(decode(y))
plt.title(decode(y))

"""



  
def Conv2d_BN(x, nb_filter,kernel_size, strides=(1,1), padding='same',name=None):  
    if name is not None:  
        bn_name = name + '_bn'  
        conv_name = name + '_conv'  
    else:  
        bn_name = None  
        conv_name = None  
  
    x = Conv2D(nb_filter,kernel_size,padding=padding,strides=strides,activation='relu',name=conv_name)(x)  
    x = BatchNormalization(axis=3,name=bn_name)(x)  
    return x  
  
def Conv_Block(inpt,nb_filter,kernel_size,strides=(1,1), with_conv_shortcut=False):  
    x = Conv2d_BN(inpt,nb_filter=nb_filter,kernel_size=kernel_size,strides=strides,padding='same')  
    x = Conv2d_BN(x, nb_filter=nb_filter, kernel_size=kernel_size,padding='same')  
    if with_conv_shortcut:  
        shortcut = Conv2d_BN(inpt,nb_filter=nb_filter,strides=strides,kernel_size=kernel_size)  
        x = add([x,shortcut])  
        return x  
    else:  
        x = add([x,inpt])  
        return x  
  
inpt = Input(shape=(224,224,3))  
x = ZeroPadding2D((3,3))(inpt)  
x = Conv2d_BN(x,nb_filter=64,kernel_size=(7,7),strides=(2,2),padding='valid')  
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)  
#(56,56,64)  
x = Conv_Block(x,nb_filter=64,kernel_size=(3,3))  
x = Conv_Block(x,nb_filter=64,kernel_size=(3,3))  
x = Conv_Block(x,nb_filter=64,kernel_size=(3,3))  
#(28,28,128)  
x = Conv_Block(x,nb_filter=128,kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True)  
x = Conv_Block(x,nb_filter=128,kernel_size=(3,3))  
x = Conv_Block(x,nb_filter=128,kernel_size=(3,3))  
x = Conv_Block(x,nb_filter=128,kernel_size=(3,3))  
#(14,14,256)  
x = Conv_Block(x,nb_filter=256,kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True)  
x = Conv_Block(x,nb_filter=256,kernel_size=(3,3))  
x = Conv_Block(x,nb_filter=256,kernel_size=(3,3))  
x = Conv_Block(x,nb_filter=256,kernel_size=(3,3))  
x = Conv_Block(x,nb_filter=256,kernel_size=(3,3))  
x = Conv_Block(x,nb_filter=256,kernel_size=(3,3))  
#(7,7,512)  
x = Conv_Block(x,nb_filter=512,kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True)  
x = Conv_Block(x,nb_filter=512,kernel_size=(3,3))  
x = Conv_Block(x,nb_filter=512,kernel_size=(3,3))  
x = AveragePooling2D(pool_size=(7,7))(x)  
x = Flatten()(x)  
#x = Dense(1000,activation='softmax')(x)  
x = [Dense(n_class, activation='softmax', name='P%d'%(i+1))(x) for i in range(7)]
model = Model(inputs=inpt,outputs=x)  
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])  
model.summary()  

#训练的时候每轮1000个样本共5轮，一个batch_size=32，所以一共有16W张图片
model.fit_generator(gen(), samples_per_epoch=1000,nb_epoch=5, 
                    nb_worker=1, pickle_safe=True, 
                    validation_data=gen(), nb_val_samples=1280)




In [0]:

!pip install graphviz
!sudo apt-get install graphviz
!pip install pydot 
#保存模型图
from keras.utils import plot_model
plot_model(model, to_file='model.png',show_shapes='True')

In [0]:
#测试下结果
X, y = next(gen(1))
y_pred = model.predict(X)
print(X)
print("ddddd")
print(X[0])
print(decode(y))
print(decode(y_pred))
plt.title('real: %s\npred:%s'%(decode(y), decode(y_pred)))
plt.imshow(X[0], cmap='gray')
#这里显示是和正常的颜色不一样，这是因为，plt读取的通道顺序和cv2的通道顺序是不同的


In [0]:
print(len(X))
print(y)

In [0]:
from google.colab import drive
# 使用工具colab的接口挂载google drive目录，这样可以从外部获取数据并且可以把训练好的模型保存在google drive上
#保存模型保存格式h5
model.save('resnet34_model.h5')

In [0]:
#分数很低是因为，评分的方式不同，用model.evaluate()函数评分是按单个字符来评分，可以更改评分的方式
score=model.evaluate(X,y,verbose=0)
print("test score=",score[0])
print("test accuracy=",score[1])